In [6]:
import pandas as pd 
# !pip install gingerit
# !pip install cloudscraper
# !pip install sentence-transformers
from sentence_transformers import SentenceTransformer

In [2]:
df = pd.read_csv('./train.csv')

In [3]:
df[df['id']=='e3b8bc8940f2a818']['comment_text']

138234    Oaks and Homophobia\nI provided a full citatio...
Name: comment_text, dtype: object

In [4]:
# df.sample(30)

In [5]:
df = (df[(df['toxic']==1) | (df['severe_toxic']==1) | (df['obscene']==1) | (df['threat']==1) | \
       (df['insult']==1) | (df['identity_hate']==1)])

In [6]:
df.shape

(16225, 8)

In [7]:
# df.sample(20)

In [8]:
import requests
import cloudscraper
URL = "https://services.gingersoftware.com/Ginger/correct/jsonSecured/GingerTheTextFull"  # noqa
API_KEY = "6ae0c3a0-afdc-4532-a810-82ded0054236"


class GingerIt(object):
    def __init__(self):
        self.url = URL
        self.api_key = API_KEY
        self.api_version = "2.0"
        self.lang = "US"

    def parse(self, text, verify=True):
        session = cloudscraper.create_scraper()
        request = session.get(
            self.url,
            params={
                "lang": self.lang,
                "apiKey": self.api_key,
                "clientVersion": self.api_version,
                "text": text,
            },
            verify=verify,
        )
        data = request.json()
        return self._process_data(text, data)

    @staticmethod
    def _change_char(original_text, from_position, to_position, change_with):
        return "{}{}{}".format(
            original_text[:from_position], change_with, original_text[to_position + 1 :]
        )

    def _process_data(self, text, data):
        result = text
        corrections = []

        for suggestion in reversed(data["Corrections"]):
            start = suggestion["From"]
            end = suggestion["To"]

            if suggestion["Suggestions"]:
                suggest = suggestion["Suggestions"][0]
                result = self._change_char(result, start, end, suggest["Text"])

                corrections.append(
                    {
                        "start": start,
                        "text": text[start : end + 1],
                        "correct": suggest.get("Text", None),
                        "definition": suggest.get("Definition", None),
                    }
                )

        return {"text": text, "result": result, "corrections": corrections}
    
    
g = GingerIt()
g.parse('helo')

{'text': 'helo',
 'result': 'Hello',
 'corrections': [{'start': 0,
   'text': 'helo',
   'correct': 'Hello',
   'definition': 'used as a greeting'}]}

In [9]:
df.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
57976,9b3b3d1e84ea0246,It also could forward to male homosexual orgie...,1,0,0,0,0,0
49057,83281a3656cbe467,FIRST YOU MURDER JESUS. NOW YOU CRUCIFY THE PA...,1,0,0,0,0,0
22730,3c041532a25f96d5,Go ahead try and block me! Just try it Drakhan!,1,0,0,0,0,0
35592,5f1e6493fe46db25,You are full of (sh)it. \n\nWhat is blatent or...,1,0,1,0,1,0
19184,32aaf210427f24a8,YOU'RE SO WELCOME- SMOKE WEED ERRRRRRRRYYYYYDA...,1,0,0,0,0,0


In [10]:
df[df['id']=='252ee9942ba7a338']['comment_text']

14101    sup niggaz \n\ncan u unprotect so i can post l...
Name: comment_text, dtype: object

In [9]:
import re 
def remove_waste(input_text):
    
    
    punct = re.compile(r'(\w+)')

    
    tokenized = [m.group() for m in punct.finditer(input_text)]
    input_text = ' '.join(tokenized)
     
    
    
    if(re.findall('\d+', input_text)):
        input_text = re.sub("\d+", '', input_text)
    if('\n' in input_text):
        input_text = input_text.replace('\n', '')
        
#     if('!' in input_text):
#         input_text = input_text.replace('!', '')
        
    if('=' in input_text):
        input_text = input_text.replace('=', '')
        
#     if("''" in input_text):
#         input_text = input_text.replace("''", '')
        
    if(")" in input_text or '(' in input_text):
        input_text = input_text.replace("(", '')
        input_text = input_text.replace(")", '')
        
    return input_text

In [15]:
import re
import string
def  clean_text(text):
    text =  text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation)) 
    text = re.sub("(\\W)"," ",text) 
    text = re.sub('\S*\d\S*\s*','', text)
    
    return text

In [16]:
# df['comment_text'] = df['comment_text'].str.lower()
df['comment_text'] = df['comment_text'].apply(clean_text)

In [17]:
df.sample(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
48380,8154e81593902da3,drop dead you are a cancer on wikipedia and i...,1,0,0,0,0,0
23303,3d8d1b373b2b9ef6,hey bitch stop taking down our fucking event p...,1,0,1,0,1,1
114961,66c6426e1d8cf745,you are making a fool of yourself this is not...,1,0,0,0,0,0
151504,7e3645ab72b9f8eb,ok listen airplane guy how dare you do u hja...,1,0,0,0,0,0
83381,df18782baef37592,do not be deliberately dense unless of course ...,1,0,0,0,0,0
147515,3d876ef81c8cc38c,old redneck jokes the name the point of the ...,1,0,0,0,0,0
135541,d4dc571558fad82c,the guy sounds like a def loser y do u care so...,1,0,0,0,1,0
46155,7b50a142ad5a428d,so fuck off and die rape fantasising sluts,1,1,1,1,1,1
36844,62627704f85947cf,so you admit youre a fag,1,0,0,0,1,0
106743,3ae0b044c136fc42,son of a bitch i will revert your edits on ter...,1,0,1,0,1,0


In [18]:
df['flag_toxic'] = 1

In [19]:
df.sample(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,flag_toxic
65897,b0337c2a0bde8378,well i suppose i spoke with a wrong person y...,1,1,1,0,1,0,1
51171,88eb9467c820118a,your space bar is broken or does it only refus...,0,0,0,0,1,0,1
109241,48320a741bf7b1b3,you edit you edit is vandal stop or die,1,0,0,1,0,0,1
47736,7f8693bec846cb25,chairmanofall once again you brainless child ...,1,0,0,0,0,0,1
98742,103534495273980e,whats your problem people have offered a chan...,1,0,0,0,0,0,1
34143,5b1c87e9f9582ad8,personal attacks regarding your last post t...,1,0,0,0,0,0,1
44390,7690e7c6dd984092,get a fucking life mate or a job you are austr...,1,0,1,0,0,0,1
16895,2c929985c8d1f460,one thing i hate is people who talk about othe...,1,0,0,0,0,0,1
14875,2750cc9f61f31cbf,sorry about the editing about max hohar but it...,1,0,1,0,1,1,1
134599,cfd580281d70d6ee,are you gonna cry and tell wikipedia suck it u...,1,0,1,0,1,0,1


In [20]:
df.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis = 1, inplace=True)

In [21]:
df

,id,comment_text,flag_toxic
6,0002bcb3da6cb337,cocksucker before you piss around on my work,1
12,0005c987bdfc9d4b,hey what is it talk what is it an exclusive...,1
16,0007e25b2121310b,bye do not look come or think of comming bac...,1
42,001810bf8c45bf5f,you are gay or antisemmitian archangel white...,1
43,00190820581d90ce,fuck your filthy mother in the ass dry,1
...,...,...,...
159494,fef4cf7ba0012866,our previous conversation you fucking shi...,1
159514,ff39a2895fc3b40e,you are a mischievious pubic hair,1
159541,ffa33d3122b599d6,your absurd edits your absurd edits on great...,1
159546,ffb47123b2d82762,hey listen do not you ever delete my edits e...,1


In [8]:
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [23]:
query = df['comment_text'].tolist()

In [24]:
query_embeddings = embedder.encode(query)

In [1]:

# with open('query_embeddings_clean.pickle', 'wb') as file:
#     pickle.dump(query_embeddings, file)
#     file.close()

In [2]:
import pickle 
with open('query_embeddings_clean.pickle', 'rb') as file:
    
    query_embeddings = pickle.load(file)

In [3]:
import scipy

In [18]:


def check(value):
    corpus_embeddings = embedder.encode(value)
    distances = scipy.spatial.distance.cdist(query_embeddings, [corpus_embeddings], "cosine")[0]
    print(1-distances)
    if(1-distances>=0.5):
        x = 'The above statement needs to be flagged'
        return x
    
    else:
        x = 'Nothing wrong here'
        return x
    
    
value = check('It was a good day !')


[0.05024188]


In [19]:
value = check('It was a good day !')
value

[0.05024188]


'Nothing wrong here'

In [37]:
value = check('Shut up you imbecile !')
value

[0.62924553]


'The above statement needs to be flagged'

In [38]:
value = check('take your dumb @$$ away from here')
value

[0.55886211]


'The above statement needs to be flagged'

In [41]:
value = check('I prefer supporting the usual norms rather than doing something revolutionary')
value

[0.29671097]


'Nothing wrong here'

In [34]:
# !pip install gradio
value = check('shut up u moron')
value

[0.56890382]


'The above statement needs to be flagged'

In [33]:
value = check('fuck you moron')
value

[0.60008562]


'The above statement needs to be flagged'

In [31]:
# import gradio as gr

# iface = gr.Interface(fn=check, inputs=["text"], outputs=["text"])

In [32]:
# !pip install anyio==3 starlette